### Regression Tree

In [124]:
import numpy as np
import sys

### Get data from txt files

In [125]:
def get_data():
    train_data = np.genfromtxt('./data/housing_train.txt')
    test_data = np.genfromtxt('./data/housing_test.txt')

    return train_data, test_data

In [126]:
# get data from the txt files
train, test = get_data()

### Normalization using shift/scale

Normalization is done on both test and train data.

In [168]:
def normalize(data_set, train_len):
    # normalize data using shift/scale
    maxs = np.max(data_set, axis=0)
    mins = np.min(data_set, axis=0)

    maxs = maxs[:-1]
    mins = mins[:-1]

    for feature in range(len(data_set[0])-1):
        for entry in data_set:
            entry[feature] = (entry[feature] - mins[feature]) / (maxs[feature] - mins[feature])

    return data_set[:train_len], data_set[train_len:]

In [169]:
# combine test and train data to perform normalization
full_data = np.concatenate((train, test), axis=0)

# normalize the data
train, test = normalize(full_data, len(train))

In [170]:
def get_mean_labels(dataset):
    return np.mean(dataset, axis=0)

### Get all possible Thresholds

Returns thresholds for all features by calculating midpoints for each feature value

In [171]:
def get_thresholds(dataset):

    ts = []
    for feature in range(len(dataset[0])-1):
        t = []
        for entry in range(len(dataset) - 1):
            t.append((dataset[entry][feature] + dataset[entry+1][feature])/2)
        ts.append(t)

    return np.array(ts)

### Get best **Feature** and **Threshold** pair for Splitting

In [179]:
def get_best_split(dataset, thresholds):
    max_mse = -sys.maxsize
    
    mse_before = get_mse(dataset)
    
    for feature in range(len(dataset[0])-1):
        for threshold in thresholds[feature]:
            left, right = split_data(dataset, feature, threshold)
            
            if len(left) <= 4 or len(right) <= 4:
                continue
                
            left_mse = get_mse(left)
            right_mse = get_mse(right)
            
            w = len(left) / len(dataset)
            mse_after = mse_before - (w * left_mse + (1-w) * right_mse)

            if max_mse <= mse_after:
                max_mse = mse_after
                best_feature = feature
                best_threshold = threshold

    return best_feature, best_threshold, max_mse

### Terminal Nodes

Represents the leaf nodes of the Regression Tree that returns the actual predictions by calculating mean label over all datapoints in the node.

In [180]:
class Terminal:

    def __init__(self, dataset):
        predicts = get_mean_labels(dataset)
        self.prediction = predicts[-1]

    def predict(self):
        return self.prediction

### Nodes

Represents a single Node in the Regression Tree which contains the splitting criteria denoted by a (Feature, Threshold) pair.

Also, contains a link to the left and right node.

In [181]:
class Node:

    def __init__(self, feature, threshold, left_node, right_node):
        self.feature = feature
        self.threshold = threshold
        self.left_node = left_node
        self.right_node = right_node

### Mean Squared Error

In [182]:
def get_mse(dataset):
    if len(dataset) == 0:
        return 0

    prediction = get_mean_labels(dataset)
    mse = 0

    for entry in dataset:
        mse += np.square(entry[-1] - prediction[-1])

    return mse

### Split the data in two bins

Splits the data based on the given feature and threshold pair

In [183]:
def split_data(dataset, feature, threshold):
    left = [entry for entry in dataset if entry[feature] < threshold]
    right = [entry for entry in dataset if entry[feature] >= threshold]
    return left, right

### Building Tree Recursively

Builds the Regression Tree and returns the root of the Tree as a model.

In [184]:
def build_tree(dataset):

    thresholds = get_thresholds(dataset)
    best_feature, best_threshold, mse_gain = get_best_split(dataset, thresholds)
    
    if mse_gain == 0:
        return Terminal(dataset)
    
    left_data, right_data = split_data(dataset, best_feature, best_threshold)

    left_node = build_tree(left_data)
    right_node = build_tree(right_data)

    return Node(best_feature, best_threshold, left_node, right_node)

In [185]:
## CREATE REGRESSION TREE

model = build_tree(train)

UnboundLocalError: local variable 'best_feature' referenced before assignment

### Print Decision Tree

In [138]:
def print_tree(root):

    if isinstance(root, Terminal):
        print('Prediction: {}'.format(root.predict()))
        return

    print('({}, {})'.format(root.feature, root.threshold))

    print('Left')
    print_tree(root.left_node)
    print('Right')
    print_tree(root.right_node)

In [ ]:
## PRINT OBTAINED MODEL

print_tree(model)

### Regress

Returns the label using learned model

In [165]:
def regress(root, entry):
    
    if isinstance(root, Terminal):
        return root.predict()
    
    if entry[root.feature] < root.threshold:
        result = regress(root.left_node, entry)
    else:
        result = regress(root.right_node, entry)
        
    return result

### Testing our Regression Tree

Returns mean squared error for the learned model applied over given test data

In [166]:
def test_model(model, test_data):
    
    predictions = []
    
    for entry in test_data:
        predictions.append(regress(model, entry))
    
    print(predictions)
    
    mse = 0
    
    for i,p in enumerate(predictions):
        mse += np.square(test_data[i][-1] - p)
        
    return mse

In [167]:
final_mse = test_model(model, test)
print(final_mse)

[12.7, 16.8, 12.7, 18.3, 19.6, 27.1, 27.1, 18.9, 21.2, 22.8, 21.2, 20.9, 35.1, 27.5, 29.9, 16.1, 18.3, 20.8, 21.4, 13.5, 19.5, 18.8, 15.4, 13.8, 19.6, 24.0, 24.3, 19.6, 33.1, 32.7, 29.0, 33.1, 33.1, 25.0, 19.7, 25.0, 25.0, 22.2, 22.5, 28.7, 38.7, 23.3, 31.0, 26.4, 21.4, 22.3, 22.0, 24.7, 14.6, 20.2, 19.6, 20.2, 11.9, 19.2, 8.8, 16.1, 8.8, 7.2, 10.9, 11.7, 11.7, 9.5, 14.1, 14.5, 14.6, 20.8, 19.9, 16.7, 10.4, 16.2, 19.2, 21.8, 24.5, 23.1]
1370.39
